In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 파일 불러오기

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_kkm_w2v.csv')
sample = pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_sample.csv')
trans_question = sample['trans_question']
answer = sample['answer']
df

In [ ]:
def str_to_vector(emd):
    res = emd.split(' ')
    #print(emd)
    #print(len(r))
    r = []
    for a in res:
        if len(a) >2:
            r.append(a)
    if len(r) != 100:
        print(len(r))
    for i in range(len(r)):
        r[i] = r[i].replace('[', '')
        r[i] = r[i].replace(']', '')
        r[i] = r[i].replace('\n', '')
        r[i] = float(r[i])
    return np.array(r)

In [ ]:
import numpy as np
embeddings = [str_to_vector(e) for e in df['embedding']] #전체 질문에 대한 임베딩 리스트

# 정확도 확인

## word2vec / fasttext 모델 사용 시 사전 준비

In [ ]:
# 역번역 질문을 공백 기준으로 나눠 리스트로 변환
##그 과정에서 오류가 나기도 해서 예외처리 해줌
t_question = []
for i in range(len(trans_question)):
  try:
    trans_question[i].split(' ')
    t_question.append(trans_question[i])
  except:
    pass
print(len(t_question))

10000


In [ ]:
for l in t_question:
    while True:
        try:
            l.remove('')
        except:
            break
print(t_question[:3])

[['구', '지방', '세법', '규정', '주주', '때', '니까'], ['형사', '소송법', '형사', '절차', '규칙', '정의', '방법', '절차', '위반', '피고', '진술', '설명', '문서', '실제', '진정', '증명', '습니까'], ['합법적', '인', '통지', '후', '도', '의장', '이사회', '참석', '니까']]


In [ ]:
from gensim.models import Word2Vec, FastText
loaded_model = Word2Vec.load('/content/drive/MyDrive/파일위치/book_law_kkm_w2v.model')
#loaded_model = FastText.load('/content/drive/MyDrive/파일위치/book_law_kkm_fst.model')

In [ ]:
#단어 벡터의 평균으로 문장(문서) 벡터 구하기
from tqdm import tqdm
def get_document_vectors(document_list):
    document_embedding_list = []
    #각 문서에 대해서
    for line in tqdm(document_list):
        doc2vec = None
        count = 0
        for word in line.split():
            #print(word)
            try:
                w = loaded_model.wv[word]
                count += 1

                #해당 문서에 있는 모든 단어들의 벡터 값을 더한다.
                if doc2vec is None :
                    doc2vec = w
                else :
                    doc2vec = doc2vec + w
            except:
                pass

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec/count
            document_embedding_list.append(doc2vec)
        else:
            document_embedding_list.append(None)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

## sbert 모델 불러오기

In [1]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 유사도 측정(의미 검색)

In [2]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
from scipy.stats import pearsonr

def pearson_similarity(a, b):
    return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [ ]:
import numpy as np

def dist(x,y):
    return np.sqrt(np.sum((x-y)**2))

In [6]:
def dense_vector_search(query, k):
    emd = sentence_model.encode(query) #임베딩_sbert
    emd = get_document_vectors(query)[0] #임베딩_w2v, fst
    sim_scores = [cos_sim(embeddings[i], emd) for i in range(len(df))]
    index = range(0, len(df))
    pairs = zip(sim_scores, index)
    result = sorted(pairs, reverse=True)[:k] #유클리드(dist) 이용 시 reverse=False로
    return zip(*result)

## 하이브리드 검색

### 키워드 기반 검색

In [7]:
import math
import numpy as np
from typing import List
from transformers import PreTrainedTokenizer, AutoTokenizer
from collections import defaultdict

class BM25:
  def __init__(self, corpus:List[List[str]], tokenizer:PreTrainedTokenizer):
    self.tokenizer = tokenizer
    self.corpus = corpus
    self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
    self.n_docs = len(self.tokenized_corpus)
    self.avg_doc_lens = sum(len(lst) for lst in self.tokenized_corpus) / len(self.tokenized_corpus)
    self.idf = self._calculate_idf()
    self.term_freqs = self._calculate_term_freqs()

  def _calculate_idf(self):
    idf = defaultdict(float)
    for doc in self.tokenized_corpus:
      for token_id in set(doc):
        idf[token_id] += 1
    for token_id, doc_frequency in idf.items():
      idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)
    return idf

  def _calculate_term_freqs(self):
    term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
    for i, doc in enumerate(self.tokenized_corpus):
      for token_id in doc:
        term_freqs[i][token_id] += 1
    return term_freqs

  def get_scores(self, query:str, k1:float = 1.2, b:float=0.75):
    query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
    scores = np.zeros(self.n_docs)
    for q in query:
      idf = self.idf[q]
      for i, term_freq in enumerate(self.term_freqs):
        q_frequency = term_freq[q]
        doc_len = len(self.tokenized_corpus[i])
        score_q = idf * (q_frequency * (k1 + 1)) / ((q_frequency) + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
        scores[i] += score_q
    return scores

  def get_top_k(self, query:str, k:int):
    scores = self.get_scores(query)
    top_k_indices = np.argsort(scores)[-k:][::-1]
    top_k_scores = scores[top_k_indices]
    return top_k_scores, top_k_indices

b_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
bm25 = BM25(list(df['question']), b_tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


### 하이브리드 검색

In [ ]:
from collections import defaultdict
#순위 조합 함수
def reciprocal_rank_fusion(rankings:List[List[int]], k):
    rrf = defaultdict(float)
    for ranking in rankings:
        for i, doc_id in enumerate(ranking, 1):
            rrf[doc_id] += 1.0 / (k + i)
    return sorted(rrf.items(), key=lambda x: x[1], reverse=True)[:5]

In [ ]:
def hybrid_search(query, k=20):
  d, dense_search_ranking = dense_vector_search(query, 100)#의미 검색
  b, bm25_search_ranking = bm25.get_top_k(query, 100) #bm25 검색
  results = reciprocal_rank_fusion([dense_search_ranking, bm25_search_ranking], k=k) #순위 조합
  return results

## 결과 확인

In [ ]:
from tqdm import tqdm
n = len(sample)
check = 0
for i in tqdm(range(n)):
    if i % 500==0:
        print(i)
        print(check)
    results = dense_vector_search(trans_question[i]) #sbert
    results = dense_vector_search(t_question[i]) #w2v / fst
    results = hybrid_search(trans_question[i]) #하이브리드 (의미 검색 중에는 sbert 선택됨)
    d = 0 #중복 방지
    for k in range(5):
      r = df['answer'][int(results[k][0])]
      if d==0 and r == answer[i]:
        check += 1
        d = 1